In [ ]:
import time
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import openpyxl

MAX_RETRIES = 10
RETRY_DELAY = 65  # Increase delay between retries to give the server more time to process


# Date range for data fetching
#start_date = datetime(2024, 7, 26)
#end_date = datetime(2024, 7, 28)

#Get today's date
end_date = datetime.now()

#Calculate the start date (1 days before today)
start_date = end_date - timedelta(days=1)

# Output file for results
output_file = r'C:\Users\praveen\Email_Sent_Users.xlsx'


# Mapping dictionary for channel names
mapping = {
    'india': 'India',
    'philippines': 'Philippines',
    'malaysia': 'Malaysia',
    'singapore': 'Singapore',
    'gulf': 'Middle East',
    'hongkong': 'Hongkong',
    'thailand': 'Thailand',
    'indonesia': 'Indonesia',
    'vietnam': 'Vietnam',
    'rexmonster': 'India',
}

# Required columns for the final result
required_columns = [
    'India',
    'Philippines',
    'Malaysia',
    'Singapore',
    'Middle East',
    'Hongkong',
    'Thailand',
    'Indonesia',
    'Vietnam',
]

# Define the list of events to fetch
events = ["Notification Sent", "Email_Sent", "Webhook Delivered"]   # Add your events here


# Function to get data for an event
def get_event_data(event_name, date_integer):
    url = "https://in1.api.clevertap.com/1/profiles.json"           #?batch_size=20000
    payload = json.dumps({
        "event_name": event_name,
        "from": date_integer,
        "to": date_integer
    })
    headers = {
        'X-CleverTap-Account-Id': '********',
        'X-CleverTap-Passcode': '*********',
        'Content-Type': 'application/json'
    }
    retry_count = 0
    while retry_count < MAX_RETRIES:
        response = requests.post(url, headers=headers, data=payload)
        response_json = response.json()

        # Handle request still in progress
        if response_json.get('status') == 'fail' and response_json.get('code') == 2:
            retry_count += 1
            print(f"Request still in progress, please retry later. Retrying... Attempt {retry_count}")
            time.sleep(RETRY_DELAY * retry_count)
            continue
        elif response_json.get('status') == 'fail':
            print(f"Failed request with error: {response_json.get('error')}")
            return {}

        # Process data with cursor pagination
        cursor = response_json.get('cursor')
        count_country = {}

        while cursor:
            next_url = f"https://in1.api.clevertap.com/1/profiles.json?cursor={cursor}"
            response = requests.get(next_url, headers=headers)

            try:
                response_json = response.json()
                cursor = response_json.get('next_cursor')
                
                # Count events by channel
                if 'records' in response_json:
                    for record in response_json['records']:
                        channel = record.get('profileData', None)
                        if channel is not None:
                            channel_name = channel.get('channel_name', None)
                            if channel_name is not None:
                                if isinstance(channel_name, list):
                                    channel_name = ', '.join(channel_name)

                                if channel_name not in count_country:
                                    count_country[channel_name] = 1
                                else:
                                    count_country[channel_name] += 1
                else:
                    print(json.dumps(response_json))
            except ValueError:
                print(response.text)

        if count_country:
            return count_country
        else:
            retry_count += 1
            print(f"Retrying... Attempt {retry_count}")
            time.sleep(RETRY_DELAY * retry_count)
    print("Max retries reached. Unable to fetch data.")
    return {}


# Main loop to iterate over dates and events
current_date = start_date
while current_date < end_date:
    date_integer = int(current_date.strftime('%Y%m%d'))
    for event in events:
        start_time = time.time()  # Start timer
        event_data = get_event_data(event, date_integer)
        if not event_data:
            print(f"No data for event {event} on {current_date}")
            current_date += timedelta(days=1)
            continue

        # Convert event data to DataFrame
        df = pd.DataFrame(list(event_data.items()), columns=['channel_name', 'count'])
        new_rows = []
        for idx, row in df.iterrows():
            channel_names = row['channel_name'].split(', ')
            for name in channel_names:
                new_rows.append({'channel_name': name.strip().lower(), 'count': row['count']})
        new_df = pd.DataFrame(new_rows)

        # Map channel names using the mapping dictionary
        def map_channel_name(name):
            for key in mapping.keys():
                if key in name.lower():
                    return mapping[key]
            return None

        new_df['channel_name_mapped'] = new_df['channel_name'].apply(map_channel_name)

        
        # Filter and group the data
        df_filtered = new_df[new_df['channel_name_mapped'].notnull()]
        result = df_filtered.groupby('channel_name_mapped')['count'].sum().reset_index()
        result.columns = ['channel_name', 'count']
        result['event_name'] = event  # Add the event name to the result

        # Ensure all required columns are present
        for column in required_columns:
            if column not in result['channel_name'].values:
                result = pd.concat([result, pd.DataFrame({'channel_name': [column], 'count': [0], 'event_name': [event]})])

        result['Date'] = pd.to_datetime(current_date, format='%Y%m%d').date()

        result.reset_index(drop=True, inplace=True)
        result = result[['Date', 'event_name', 'channel_name', 'count']]

        end_time = time.time()  # End timer
        execution_time = end_time - start_time  # Calculate execution time

        print(result)
        print(f"Execution time for {event} on {current_date}: {execution_time:.2f} seconds")

        # Append results to Excel file
        try:
            with pd.ExcelWriter(output_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
                book = writer.book
                if 'Event_Data' in book.sheetnames:
                    existing_data = pd.read_excel(output_file, sheet_name='Event_Data')
                    if 'Date' not in existing_data.columns:
                        raise KeyError("'Date' column is missing in the existing data")
                    if 'event_name' not in existing_data.columns:
                        existing_data['event_name'] = None
                    existing_data['Date'] = pd.to_datetime(existing_data['Date'])
                    
                    # Ensure uniqueness in existing_data
                    existing_data = existing_data.drop_duplicates(subset=['Date', 'event_name', 'channel_name'])
                    result = result.drop_duplicates(subset=['Date', 'event_name', 'channel_name'])
                    updated_data = pd.concat([existing_data, result], ignore_index=True)
                    updated_data.to_excel(writer, sheet_name='Event_Data', index=False)
                else:
                    result.to_excel(writer, sheet_name='Event_Data', index=False)
        except FileNotFoundError:
            with pd.ExcelWriter(output_file, mode='w', engine='openpyxl') as writer:
                result.to_excel(writer, sheet_name='Event_Data', index=False)

    current_date += timedelta(days=1)